# Neural Network activations research

This research tries to address 3 topics for neural networks:
- test how a single layer neural network can learn a multivariate gaussian distribution
- test the impact of the type of the activation on the layer has on the result
- implement a new custom activation based on RELU and test its impact - TODO

For this research our own library was created named matmih.

### Import required libraries

In [ ]:
# We require at least tensorflow 2.3 and tensorflow-probability 0.11
import sys
import os
import warnings
%reload_ext autoreload
%autoreload

def mount_gdrive():
    # Import the library and kaggle config files from gdrive
    GDRIVE_PATH='/content/gdrive/RESEARCH'
    if 'google.colab' in sys.modules:
        from google.colab import drive
        import shutil
        drive.mount('/content/gdrive')
        sys.path.append(GDRIVE_PATH)
        os.makedirs('/root/.kaggle/', exist_ok=True)
        shutil.copyfile(GDRIVE_PATH + 'kaggle.json', '/root/.kaggle/kaggle.json')
        !chmod 600 '/root/.kaggle/kaggle.json'

def install_modules():
    !pip install --quiet kaggle
    !pip install --quiet pandas
    !pip install --quiet scikit-learn
    !pip install --quiet scikit-image
    !pip install --quiet scipy
    !pip install --quiet statsmodels
    !pip install --upgrade --quiet tensorflow
    !pip install --upgrade --quiet tensorflow-probability
    !pip install --quiet randomcolor
    !pip install --quiet matplotlib
    !pip install --quiet Pillow
    !pip install --quiet arviz
    !pip install --quiet seaborn
    !pip install --quiet prettytable

mount_gdrive()
#install_modules()

import matmih as mm
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.python.platform.build_info.build_info)

### Multivariate normal dataset creation
Created a dataset using by sampling from multivariate normal distribution for the features and using a categorical distribution for the classes. Each class will have its own mean and covariance.

Since the data distribution is know the best estimator is Bayes so we can compute the baseline best accuracy class as 
$ argmax(P(C_i) * P_{MVNdistro}(features)) $

In [ ]:
class Distr_DataSet:
    def __init__(self, class_probs, feature_distributions):
        assert(len(class_probs) == len(feature_distributions))

        self._no_classes = len(class_probs)
        self._class_choice = tfp.distributions.Categorical(probs=class_probs)
        self._distr = feature_distributions
        self._feature_shape = feature_distributions[0].event_shape
    
    def sample(self, N):
        s = np.zeros((N, *self._feature_shape))
        choice = self._class_choice.sample(N)
        for i in range(N):
            s[i] = self._distr[choice[i]].sample(1).numpy().squeeze()
        return s, choice.numpy()
    
    def bayes_classifier(self, features):
        classes = np.zeros(len(features), dtype=np.int32)
        class_scores = np.array([self._class_choice.prob(i) for i in range(self._no_classes)])
        for j, feature in enumerate(features):
            scores = np.array([self._distr[i].prob(feature) for i in range(self._no_classes)])
            classes[j] = np.argmax(class_scores * scores)
        
        return classes
            

class MVNormal_Data(Distr_DataSet):
    def __init__(self, class_prob: list, no_features):
        features = []
        for i in range(len(class_prob)):
            unif = tfp.distributions.Uniform(low=[-1] * no_features, high=[5] * no_features)
        
            loc = unif.sample()

            tril=tf.linalg.LinearOperatorLowerTriangular(unif.sample(sample_shape=(no_features,))).to_dense()

            distr = tfp.distributions.MultivariateNormalTriL(loc=loc, scale_tril=tril, name='class{}'.format(i))
            del unif

            features.append(distr)

        super(MVNormal_Data, self).__init__(class_prob, features)
        
# sample a binomial classification dataset
data = MVNormal_Data([0.5, 0.5], 5)
features, classes = data.sample(1000)
bayes_classes = data.bayes_classifier(features)
print("BAYES ACCURACY {}".format(mm.Model.accuracy(classes, bayes_classes)))

### Configurable NN model
Using matmih library we create a configurable neural network model using Keras.

All model parameters including the hidden layer size and activation as well as the train epochs, optimier and so on can be configured by passing a hyperparameter dictinary.

In [ ]:
class NNModel(mm.TensorModel):
    BATCH_SIZE = 32

    def __init__(self, **hyper_params):
        self._hyper_params = hyper_params.copy()
        no_classes = hyper_params.get('noClasses', 2)
        if no_classes == 2:
            no_classes = 1

        model = tf.keras.Sequential([
            tf.keras.layers.Dense(hyper_params.get('sizeL1', 10),
                                  input_shape=(hyper_params.get('noFeatures', 4),),
                                 name='Hidden_1'),
            tf.keras.layers.BatchNormalization(name='Hidden_1_BN'),
            tf.keras.layers.Activation(hyper_params.get('actL1', 'relu'),
                                      name='Hidden_1_Activation'),
            tf.keras.layers.Dense(no_classes, name='Output_Logits'),
            tf.keras.layers.Activation('softmax', name='softmax')
               if no_classes > 1 else
            tf.keras.layers.Activation('sigmoid', name='sigmoid')
            ], name=hyper_params.get('model_name', 'NN'))
        super(NNModel, self).__init__(model, checkpoint=False)

        self._train_epochs = hyper_params.get('trainEpochs', 20)
        self._optimizer = hyper_params.get('optimizer', tf.keras.optimizers.RMSprop())

        # compile the model and initialize the weights
        self._model.compile(
             optimizer=self._optimizer,
             loss='sparse_categorical_crossentropy' if no_classes > 1 else 'binary_crossentropy',
             metrics=['accuracy'])

    # Convert the features/target np data to a tensorflow dataset
    @staticmethod
    def np_to_tf(features, target=None, batch_size=BATCH_SIZE):
        if target is None:
            ds = tf.data.Dataset.from_tensor_slices( (tf.cast(features, tf.float32)) )
        else:
            ds = tf.data.Dataset.from_tensor_slices( (tf.cast(features, tf.float32),
                                                      tf.cast(target, tf.int32)) )

        return ds if batch_size is None else ds.batch(batch_size)

    def train(self, data_model, logTensorBoard=False):
        callbacks = []
        if logTensorBoard:
            callbacks += [tf.keras.callbacks.TensorBoard(mm.TensorBoard.get_log_dir(), histogram_freq=1)]

        train_ds = NNModel.np_to_tf(data_model.train_features, data_model.train_target)
        validation_ds = NNModel.np_to_tf(data_model.validation_features, data_model.validation_target)

        history = self._model.fit(train_ds, validation_data=validation_ds,
                                  epochs=self._train_epochs, callbacks=callbacks,
                                  verbose=self._hyper_params.get('verbose', 1))

        return mm.ModelHistory(self._hyper_params, history.history)

    def predict(self, features):
        features_ds = tf.cast(features, tf.float32)
        return self._model.predict_classes(features_ds), self._model.predict(features_ds)

# Print the summary of the model used for multiclass classification
NNModel(noFeatures=4, noClasses=3, denseSize=5)._model.summary()

### GUReLU - Gaussian Uniform ReLU
Inspired from Relu6 activation paper [http://www.cs.utoronto.ca/~kriz/conv-cifar10-aug2010.pdf] we propose a new activation where the constant 6 is replaced by a random variable.

Noise added is from a HalfNormal distribution meaning we always increase the activation by some amount.

In [ ]:
from tensorflow.python.keras.utils.generic_utils import get_custom_objects

class GU_NNModel(NNModel):
    def __init__(self, **hyper_params):
        get_custom_objects().update({
            'gurelu': tf.keras.layers.Activation(self.gurelu_activation)})

        self._threshold = tfp.distributions.Uniform(low=hyper_params.get('low_gurelu', 0.0),
                                                    high=(hyper_params.get('high_gurelu', 10.0) 
                                                    * hyper_params.get('noFeatures', 4)).astype(np.float32))
        self._noise = tfp.distributions.HalfNormal(scale=0.5)

        super(GU_NNModel, self).__init__(**hyper_params)

    @tf.custom_gradient
    def gurelu_activation(self, x):
        max_value = self._threshold.sample((1,1))

        @tf.function
        def grad(dy):
            zero_grad = tf.zeros(shape=(1, dy.shape[1]), dtype=dy.dtype)
            dy_grelu = tf.where(tf.math.less(x, 0), zero_grad, dy)
            dy_grelu = tf.where(tf.math.greater(x, max_value), zero_grad, dy)
            return dy_grelu

        min_value = tf.constant(0, shape=(1,1), dtype=x.dtype)
        x_relu = tf.clip_by_value(x, min_value, max_value)
        x_relu = tf.math.add(x_relu, self._noise.sample((1, x.shape[1])))
        return x_relu, grad

### Models training and evaluation
First we create a binary classification distribution using only 3 featured sampled from the above multivariate normal dataset class.

We train and run the models on **independent** samples from the same distribution so that we can correctly do statistical tests on the results

In [ ]:
def TRAIN_MODEL(NO_CLASSES, NO_FEATURES, ACTIVATIONS, CLASS_PROB: list, MVN_SAMPLES: list):
    HISTORY = mm.ModelHistorySet()
    PARAMS = {'noFeatures' : NO_FEATURES, 'noClasses' : NO_CLASSES, 'trainEpochs' : 100, 'verbose' : 0}
    # the gaussian multivariate distibution dataset
    DISTRIBUTION = MVNormal_Data(CLASS_PROB, NO_FEATURES)

    for i, no_samples in enumerate(MVN_SAMPLES):
        features, classes = DISTRIBUTION.sample(no_samples)
        ds = mm.DataModel(features, classes).split([0.8, 0.2])

        # Set GUReLU threshold to be the maximum feature value
        PARAMS['high_gurelu'] = np.max(ds.train_features).astype(np.float32)

        print("{}SAMPLES={} BAYES ACCURACY TRAIN={:.3f} VALIDATION={:.3f}{}".format(
            '='*20, no_samples,
            mm.Model.accuracy(ds.train_target, DISTRIBUTION.bayes_classifier(ds.train_features)),
            mm.Model.accuracy(ds.validation_target, DISTRIBUTION.bayes_classifier(ds.validation_features)),'='*20))

        for activation in ACTIVATIONS:
            for layer_size in [features.shape[1], 2*features.shape[1], 4*features.shape[1]]:
                model = GU_NNModel(model_name='NN_sample{}'.format(i), **PARAMS,
                                sizeL1=layer_size, actL1=activation)

                history = model.train(ds)
                HISTORY.add_history(history)
                print('>>>MODEL activation={} dense_layer={:2d} >>> BEST ACCURACY TRAIN={:.3f} VALIDATION={:.3f}'.format(
                    activation, layer_size,
                    max(history.history('accuracy', mm.DataType.TRAIN)),
                    max(history.history('accuracy', mm.DataType.VALIDATION))))
                model.destroy()
                del model
    return HISTORY

### Binary classification
We train the NN model against 800 samples and validate it against 200 samples taken from a multivariate normal distribution with 4 features using different activation for the single hidden layer.

Each class has equal 0.5 probability of sampling and we train the model 6 times on independent samples on the distribution

In [ ]:
ACTIVATIONS = ['gurelu', 'relu', 'tanh', 'softplus']
NO_FEATURES = 4
NO_CLASSES = 2
CLASS_PROB = np.array([1] * NO_CLASSES) / NO_CLASSES
NMV_SAMPLES = [1000, 1000, 1000, 1000, 1000, 1000, 1000]

HISTORY_2class = TRAIN_MODEL(NO_CLASSES, NO_FEATURES, ACTIVATIONS, CLASS_PROB, NMV_SAMPLES)

### Multi-class classification
We train the NN model against 800 samples and validate it against 200 samples taken from a multivariate normal distribution with 10 features using different activation for the single hidden layer.

Each class has equal 0.333 probability of sampling and we train the model 6 times on independent samples on the distribution

In [ ]:
ACTIVATIONS = ['gurelu', 'relu', 'tanh', 'softplus']
NO_FEATURES = 10
NO_CLASSES = 3
CLASS_PROB = np.array([1] * NO_CLASSES) / NO_CLASSES
NMV_SAMPLES = [1000, 1000, 1000, 1000, 1000, 1000, 1000]

HISTORY_3class = TRAIN_MODEL(NO_CLASSES, NO_FEATURES, ACTIVATIONS, CLASS_PROB, NMV_SAMPLES)

### Plot the results of the training
Use a continous line for the training set metric and dashed line for validation for each trained sample.

We split the results per activation function.

In [ ]:
EVALUATION_2 = mm.ModelEvaluation(HISTORY_2class).set_filter_params(['actL1', 'sizeL1'])

EVALUATION_2.plot_history('GURELU - 2 class', ['accuracy', 'loss'], actL1='gurelu')
EVALUATION_2.plot_history('RELU - 2 class', ['accuracy', 'loss'], actL1='relu')
EVALUATION_2.plot_history('TANH - 2 class', ['accuracy', 'loss'], actL1='tanh')
EVALUATION_2.plot_history('SOFTPLUS - 2 class', ['accuracy', 'loss'], actL1='softplus')

In [ ]:
EVALUATION_3 = mm.ModelEvaluation(HISTORY_3class).set_filter_params(['actL1', 'sizeL1'])

EVALUATION_3.plot_history('GURELU - 2 class', ['accuracy', 'loss'], actL1='gurelu')
EVALUATION_3.plot_history('RELU - 2 class', ['accuracy', 'loss'], actL1='relu')
EVALUATION_3.plot_history('TANH - 2 class', ['accuracy', 'loss'], actL1='tanh')
EVALUATION_3.plot_history('SOFTPLUS - 2 class', ['accuracy', 'loss'], actL1='softplus')

## Models evaluation
We use statistical tests to see if there is any statistical difference between **best accuracy** of the models.

To do this we this the data is represented for each model (e.g. model with activation RELU and layer size 12) as the best accuracy given by each of the train-validation accuracy result for each independent sample from the distribution.

### Models accuracy distribution
Plot the distribution of the best accuracy for each models.

In [1]:
print("{}BINARY CLASSIFICATION - 2 class{}".format('='*10, '='*10))
EVALUATION_2.plot_distributions('accuracy', mm.DataType.TRAIN)
EVALUATION_2.plot_distributions('accuracy', mm.DataType.VALIDATION)

==========BINARY CLASSIFICATION - 2 class==========


NameError: name 'EVALUATION_2' is not defined

In [ ]:
print("{}MULTICLASS CLASSIFICATION - 3 class{}".format('='*10, '='*10))
EVALUATION_3.plot_distributions('accuracy', mm.DataType.TRAIN)
EVALUATION_3.plot_distributions('accuracy', mm.DataType.VALIDATION)

### Paired statistical tests
We use a paired statistical tests between each model to check if the mean of the accurary of each is statistically different.
To do so we use 2 tests:
- Student paired t-test - parametric test that requires that the data is normal distributed.
- Wilcoxon signed rank test - non-parametric test that has no data requirement but has less power (larger type II error)

In [ ]:
print("{}BINARY CLASSIFICATION - 2 class{}".format('='*10, '='*10))
EVALUATION_2.set_p_threshold(0.01)
EVALUATION_2.paired_statistical_test('accuracy', mm.DataType.TRAIN)

In [ ]:
EVALUATION_2.paired_statistical_test('accuracy', mm.DataType.VALIDATION)

In [ ]:
print("{}MULTICLASS CLASSIFICATION - 3 class{}".format('='*10, '='*10))
EVALUATION_3.set_p_threshold(0.01)
EVALUATION_3.paired_statistical_test('accuracy', mm.DataType.TRAIN)

In [ ]:
EVALUATION_3.paired_statistical_test('accuracy', mm.DataType.VALIDATION)

### Anova one-way tests
We use anova to check that the mean accuracy of all models (each model is a group) is the same or not.

Anova requires the group variances (model accuracies) to be equal so we also perform a Bartlett covariance test.

In [ ]:
print("{}BINARY CLASSIFICATION - 2 class{}".format('='*10, '='*10))
EVALUATION_2.oneway_anova_test('accuracy', mm.DataType.TRAIN)
EVALUATION_2.oneway_anova_test('accuracy', mm.DataType.VALIDATION)

In [ ]:
print("{}MULTICLASS CLASSIFICATION - 3 class{}".format('='*10, '='*10))
EVALUATION_3.oneway_anova_test('accuracy', mm.DataType.TRAIN)
EVALUATION_3.oneway_anova_test('accuracy', mm.DataType.VALIDATION)

### Tukey honestly significant difference test
Anova has the problem that it can only tell use if there is a difference between the mean of all the tests

We use Tukey HSD to do pairwise tests same as in the Student t-test.

In [ ]:
print("{}BINARY CLASSIFICATION - 2 class{}".format('='*10, '='*10))
EVALUATION_2.tukey_hsd_test('accuracy', mm.DataType.TRAIN)
EVALUATION_2.tukey_hsd_test('accuracy', mm.DataType.VALIDATION)

In [ ]:
print("{}MULTICLASS CLASSIFICATION - 3 class{}".format('='*10, '='*10))
EVALUATION_3.tukey_hsd_test('accuracy', mm.DataType.TRAIN)
EVALUATION_3.tukey_hsd_test('accuracy', mm.DataType.VALIDATION)